In [ ]:
# Tutorial 2: Single cell simulation with external feedfoward input (with BioNet)

In [ ]:
from bmtk.builder.networks import NetworkBuilder

# 构建单个细胞组成的网络，步骤与Tutorial1相同
# 实例化
cortex = NetworkBuilder('mcortex')

# 添加单个节点
cortex.add_nodes(
    cell_name='Scnn1a_473845048',
    potental='exc',
    model_type='biophysical',
    model_template='ctdb:Biophys1.hoc',
    model_processing='aibs_perisomatic',
    dynamics_params='472363762_fit.json',
    morphology='Scnn1a_473845048_m.swc'
)

# 构建
cortex.build()

# 保存
cortex.save_nodes(output_dir='sim_ch02/network')

In [ ]:
# 构建新网络，这是一个虚拟细胞，没有正常神经元的形态与特性，此处只充当一个脉冲发射器，表示来自丘脑的输入
# 实例化
thalamus = NetworkBuilder('mthalamus')

# 添加10个节点（N=10）
thalamus.add_nodes(
    N=10,
    pop_name='tON',
    potential='exc',
    model_type='virtual'  # 不再是生物物理细胞（'biophysical'），而是虚拟细胞（'virtual'）
)

In [ ]:
# 连接两个网络，方法是添加边（edges）
thalamus.add_edges(
    # 设置source和target
    source={'pop_name': 'tON'},   # 设置起始节点
    # source=thalamus.nodes(),  # 这样写也可以
    target=cortex.nodes(),  # 设置目标节点
    
    # 设置突触个数
    connection_rule=5,  # 表示source和target节点之间有5个突触
    
    # 权重
    syn_weight=0.001,  # 连接强度
    delay=2.0,  # 连接延迟是2ms
    weight_function=None,  # 用于调整权重
    
    # 决定在突触后神经元的哪个位置设置突触，默认情况下是在给定部分和距离范围内随机
    target_sections=['basal', 'apical'],  # 给定部分
    distance_range=[0.0, 150.0],  # 距离范围
    
    # 参数文件
    dynamics_params='AMPA_ExcToExc.json',
    model_template='exp2syn'
)

In [ ]:
# 构建
thalamus.build()

# 保存节点
thalamus.save_nodes(output_dir='sim_ch02/network')

# 保存边
thalamus.save_edges(output_dir='sim_ch02/network')

In [ ]:
# 因为节点文件和边文件分开来存储，所以后续重建、修改等工作就方便很多了

In [ ]:
from bmtk.utils.reports.spike_trains import PoissonSpikeGenerator

# 建立尖峰队列（Spike Train），按泊松分布（Poisson）
psg = PoissonSpikeGenerator(population='mthalamus')
psg.add(
    node_ids=range(10),  # 10个与mthalamus（先前创建的虚拟细胞）对应的节点
    firing_rate=10.0,    # 发放的速率为10赫兹，这里也可以传入函数或数组
    times=(0.0, 3.0)    # 发放从0秒到3秒开始
)

# 保存为Sonata格式的队列（除了.h5以外，BMTK也允许存为csv或NWB文件）
psg.to_sonata('sim_ch02/inputs/mthalamus_spikes.h5')

In [ ]:
# 打印参数
print('Number of spikes: {}'.format(psg.n_spikes()))
print('Units: {}'.format(psg.units()))

psg.to_dataframe().head()

In [ ]:
# Step 2: Setting up BioNet environment.
# 在运行模拟器之前，需要先创建运行时环境，即程序运行时为其提供所需资源的软件环境，包括操作系统、库函数、配置文件等

In [ ]:
from bmtk.utils.sim_setup import build_env_bionet

# 参数与Tutorial1中学习到的差不多，此处不写注释（保留原文档给出的英文注释）
build_env_bionet(
    base_dir='sim_ch02',
    config_file='config.json',
    network_dir='sim_ch02/network',
    tstop=3000.0, dt=0.1,
    report_vars=['v', 'cai'],    # Record membrane potential and calcium (default soma)
    spikes_inputs=[('mthalamus', # Name of population which spikes will be generated for
                    'sim_ch02/inputs/mthalamus_spikes.h5')],
    include_examples=True,       # Copies components files
    compile_mechanisms=True      # Will try to compile NEURON mechanisms
)

In [ ]:
from bmtk.simulator import bionet

# 运行模拟器
conf = bionet.Config.from_json('sim_ch02/config.json')
conf.build_env()
net = bionet.BioNetwork.from_config(conf)
sim = bionet.BioSimulator.from_config(conf, network=net)
sim.run()

# 打印
print('Success!')

In [ ]:
from bmtk.analyzer.spike_trains import to_dataframe

# 分析运行结果
results_df = to_dataframe(config_file='sim_ch02/config.json')
print('Number of Spikes: {}'.format(len(results_df)))
results_df.head()

In [ ]:
from bmtk.analyzer.compartment import plot_traces

_ = plot_traces(config_file='sim_ch02/config.json', node_ids=[0], report_name='v_report')
_ = plot_traces(config_file='sim_ch02/config.json', node_ids=[0], report_name='cai_report')